In [1]:
import cv2
import os
import random
import torch
from ultralytics import YOLO
import os
import shutil
import xml.etree.ElementTree as ET



In [23]:
import cv2
import os

# Set video path
video_path = "Rams-Saints One.mp4"  # Replace with your video file
output_folder = "extracted_frames"

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Open video file
cap = cv2.VideoCapture(video_path)

frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # Stop if video ends

    # Save frame as an image
    frame_filename = os.path.join(output_folder, f"frame_{frame_count:04d}.jpg")
    cv2.imwrite(frame_filename, frame)

    frame_count += 1

cap.release()
print(f"Extracted {frame_count} frames and saved in '{output_folder}'")

Extracted 183 frames and saved in 'extracted_frames'


In [24]:
import cv2
import os
from ultralytics import YOLO

# Load YOLO model (Replace "best.pt" with your actual model)
model = YOLO("datasets/final-training/runs/detect/train/weights/best.pt")  

# Folder containing extracted frames
input_folder = "extracted_frames"
output_folder = "yolo_output_frames"

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Get all frame filenames
frame_files = sorted([f for f in os.listdir(input_folder) if f.endswith(".jpg")])

for frame_file in frame_files:
    frame_path = os.path.join(input_folder, frame_file)
    frame = cv2.imread(frame_path)

    # Run YOLO inference
    results = model(frame)

    # Draw detections
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding box coordinates
            conf = box.conf[0].item()  # Confidence score
            cls = int(box.cls[0].item())  # Class ID

            label = f"{model.names[cls]} {conf:.2f}"
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Save frame with detections
    output_path = os.path.join(output_folder, frame_file)
    cv2.imwrite(output_path, frame)

print(f"Processed {len(frame_files)} frames. Output saved in '{output_folder}'")



0: 384x640 (no detections), 40.8ms
Speed: 4.2ms preprocess, 40.8ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 24.9ms
Speed: 1.4ms preprocess, 24.9ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 24.8ms
Speed: 1.7ms preprocess, 24.8ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 24.9ms
Speed: 1.5ms preprocess, 24.9ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 25.0ms
Speed: 1.7ms preprocess, 25.0ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 15.2ms
Speed: 1.5ms preprocess, 15.2ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 15.2ms
Speed: 1.4ms preprocess, 15.2ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 15.3ms
Speed: 1.3ms preprocess, 15.3ms i

In [25]:
def create_video_from_frames(frame_folder, output_video="output.mp4", fps=30):
    frames = [f for f in os.listdir(frame_folder) if f.endswith(".jpg")]
    frames.sort()  # Ensure frames are in correct order

    if not frames:
        print("No frames found in the folder.")
        return

    first_frame = cv2.imread(os.path.join(frame_folder, frames[0]))
    height, width, layers = first_frame.shape

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4 format
    out = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

    for frame in frames:
        img = cv2.imread(os.path.join(frame_folder, frame))
        out.write(img)

    out.release()
    print(f"Video saved as {output_video}")

# Example Usage
create_video_from_frames("yolo_output_frames", "interference_detection.mp4", fps=30)

Video saved as interference_detection.mp4
